In [1]:
# Imports
import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime
from pathlib import Path

# MLflow
import mlflow
import mlflow.sklearn
import dagshub
from dotenv import load_dotenv

# ML Models
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Tuning
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)

import warnings
warnings.filterwarnings('ignore')

print("✅ Imports terminés")

✅ Imports terminés


In [2]:
import os
from dotenv import load_dotenv
import mlflow
import dagshub

# Configuration MLflow + DagsHub
load_dotenv()

DAGSHUB_USERNAME = os.getenv('DAGSHUB_USERNAME')
DAGSHUB_TOKEN = os.getenv('DAGSHUB_TOKEN')
DAGSHUB_REPO = os.getenv('DAGSHUB_REPO_NAME')

MLFLOW_TRACKING_URI = f"https://dagshub.com/{DAGSHUB_USERNAME}/{DAGSHUB_REPO}.mlflow"
EXPERIMENT_NAME = "Crime_MLOPS"

# Configuration DagsHub
dagshub.init(repo_owner=DAGSHUB_USERNAME, repo_name=DAGSHUB_REPO, mlflow=True)

# Configuration MLflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

# Désactiver les features non supportées par DagsHub
os.environ['MLFLOW_ENABLE_LOGGED_MODEL_CREATION'] = 'false'

if DAGSHUB_TOKEN:
    os.environ['MLFLOW_TRACKING_USERNAME'] = DAGSHUB_USERNAME
    os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN

print("✅ MLflow configuré")
print(f"📊 Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"🧪 Experiment: {EXPERIMENT_NAME}" )

Accessing as ImenBenAmar

Initialized MLflow to track repo "YomnaJL/MLOPS_Project"

Repository YomnaJL/MLOPS_Project initialized!

✅ MLflow configuré
📊 Tracking URI: https://dagshub.com/YomnaJL/MLOPS_Project.mlflow
🧪 Experiment: Crime_MLOPS


In [8]:
import os
import time
import pickle
import getpass
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.notebook import tqdm  # Interactive progress bar

import mlflow
import mlflow.sklearn
import mlflow.xgboost
import mlflow.lightgbm
import mlflow.catboost
import dagshub

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, precision_score, recall_score,
    confusion_matrix
)

# Load environment variables
load_dotenv()

# --- Configuration ---
DAGSHUB_USERNAME = os.getenv('DAGSHUB_USERNAME')
DAGSHUB_TOKEN = os.getenv('DAGSHUB_TOKEN')
DAGSHUB_REPO = os.getenv('DAGSHUB_REPO_NAME')
EXPERIMENT_NAME = "Crime_MLOPS"
DATA_PATH = '../processors/preprocessed_data.pkl'

# Initialize DagsHub & MLflow
if all([DAGSHUB_USERNAME, DAGSHUB_TOKEN, DAGSHUB_REPO]):
    MLFLOW_TRACKING_URI = f"https://dagshub.com/{DAGSHUB_USERNAME}/{DAGSHUB_REPO}.mlflow"
    
    dagshub.init(repo_owner=DAGSHUB_USERNAME, repo_name=DAGSHUB_REPO, mlflow=True)
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment(EXPERIMENT_NAME)
    
    # Auth setup
    os.environ['MLFLOW_TRACKING_USERNAME'] = DAGSHUB_USERNAME
    os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN
    os.environ['MLFLOW_ENABLE_LOGGED_MODEL_CREATION'] = 'false'
    
    print(f"✅ MLflow Configured via DagsHub")
    print(f"📊 Tracking URI: {MLFLOW_TRACKING_URI}")
else:
    print("⚠️ Missing .env variables. Running locally only.")



Initialized MLflow to track repo "YomnaJL/MLOPS_Project"

Repository YomnaJL/MLOPS_Project initialized!

✅ MLflow Configured via DagsHub
📊 Tracking URI: https://dagshub.com/YomnaJL/MLOPS_Project.mlflow


In [6]:
# User Tagging
user_name = input("Enter your name: ")
print(f"👤 User: {user_name}")

👤 User: imen


In [9]:
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Data file '{DATA_PATH}' not found. Run preprocessing first.")

with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train_scaled']
X_test = data['X_test_scaled']
y_train = data['y_train']
y_test = data['y_test']

print(f"✅ Data Loaded Successfully")
print(f"   Train Shape: {X_train.shape}")
print(f"   Test Shape:  {X_test.shape}")

✅ Data Loaded Successfully
   Train Shape: (319110, 17)
   Test Shape:  (79778, 17)


In [10]:
# Global Leaderboard DataFrame
leaderboard = pd.DataFrame(columns=["Stage", "Model", "Accuracy", "F1_Weighted", "ROC_AUC", "Time (s)"])

def log_confusion_matrix(y_true, y_pred, model_name, stage, show_plot=False):
    """Generates confusion matrix, saves to MLflow, and optionally shows it."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {model_name} ({stage})')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    filename = f"confusion_matrix_{model_name}_{stage}.png"
    plt.savefig(filename)
    
    if show_plot:
        plt.show()
    else:
        plt.close()
    
    mlflow.log_artifact(filename)
    if os.path.exists(filename):
        os.remove(filename)

def train_evaluate_log(model_name, model, stage, params=None, tune=False, param_dist=None, n_iter=6):
    """
    Main pipeline function: Train/Tune -> Predict -> Evaluate -> Log to MLflow
    """
    global leaderboard
    
    with mlflow.start_run(run_name=f"{model_name}_{stage}"):
        # 1. Tags
        mlflow.set_tag("user", user_name)
        mlflow.set_tag("dataset_version", "v1")
        mlflow.set_tag("model_name", model_name)
        mlflow.set_tag("stage", stage)
        
        final_model = model
        tune_time = 0
        
        # 2. Tuning (Optional)
        if tune and param_dist:
            print(f"   🔧 Tuning {model_name} (n_iter={n_iter})...")
            start_tune = time.time()
            search = RandomizedSearchCV(
                estimator=model,
                param_distributions=param_dist,
                n_iter=n_iter,
                scoring='f1_weighted',
                cv=3,
                verbose=0,
                random_state=42,
                n_jobs=-1
            )
            search.fit(X_train, y_train)
            tune_time = time.time() - start_tune
            
            final_model = search.best_estimator_
            params = search.best_params_
            
            mlflow.log_metric("tuning_time_seconds", tune_time)
            mlflow.log_params(params)
            print(f"      ✓ Tuning Time: {tune_time:.2f}s")
        elif params:
            mlflow.log_params(params)

        # 3. Training
        start_train = time.time()
        if not tune:
             final_model.fit(X_train, y_train)
        train_time = time.time() - start_train
        
        # 4. Predictions
        start_pred = time.time()
        y_pred = final_model.predict(X_test)
        pred_time = time.time() - start_pred
        
        try:
            y_prob = final_model.predict_proba(X_test)
        except AttributeError:
            y_prob = None

        # 5. Metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        
        roc_auc = 0
        if y_prob is not None:
             try:
                roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='weighted')
             except:
                pass

        # 6. MLflow Logging
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision_weighted", prec)
        mlflow.log_metric("recall_weighted", rec)
        mlflow.log_metric("f1_weighted", f1)
        mlflow.log_metric("roc_auc_ovr_weighted", roc_auc)
        
        if not tune:
             mlflow.log_metric("training_time_seconds", train_time)
        mlflow.log_metric("prediction_time_seconds", pred_time)

        # Log Model
        if "XGBoost" in model_name: mlflow.xgboost.log_model(final_model, "model")
        elif "LightGBM" in model_name: mlflow.lightgbm.log_model(final_model, "model")
        elif "CatBoost" in model_name: mlflow.catboost.log_model(final_model, "model")
        else: mlflow.sklearn.log_model(final_model, "model")

        # Log Confusion Matrix
        log_confusion_matrix(y_test, y_pred, model_name, stage)
        
        # Update Leaderboard
        new_row = pd.DataFrame([{
            "Stage": stage,
            "Model": model_name,
            "Accuracy": acc,
            "F1_Weighted": f1,
            "ROC_AUC": roc_auc,
            "Time (s)": train_time + tune_time
        }])
        leaderboard = pd.concat([leaderboard, new_row], ignore_index=True)
        
        return final_model

In [ ]:
# --- Baseline Configurations ---
base_models_config = {
    "RandomForest": RandomForestClassifier(
        n_estimators=150, max_depth=15, class_weight='balanced', n_jobs=-1, random_state=42
    ),
    "XGBoost": XGBClassifier(
        n_estimators=150, learning_rate=0.1, max_depth=6, eval_metric='mlogloss', n_jobs=-1, random_state=42
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=150, learning_rate=0.1, class_weight='balanced', n_jobs=-1, random_state=42, verbose=-1
    ),
    "CatBoost": CatBoostClassifier(
        iterations=150, learning_rate=0.1, depth=6, verbose=0, random_state=42, allow_writing_files=False
    )
}

# --- Hyperparameter Grids ---
param_grids = {
    'RandomForest': { 
        'n_estimators': [100, 300, 600],
            'max_depth': [10,30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 3, 6],
            'max_features': ['sqrt', 'log2'],
            'class_weight': ['balanced', 'balanced_subsample']
    },
    'XGBoost': { 
        'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7, 10],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'gamma': [0, 0.1, 0.2],
            'min_child_weight': [1, 3, 5]
    },
    'LightGBM': { 
        'n_estimators': [100, 200, 300],
            'max_depth': [-1, 10, 20],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [20, 31, 50, 100],
            'min_child_samples': [10, 20, 30],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'class_weight': ['balanced', None]
    },
    'CatBoost': { 
        'iterations': [100, 200, 300],
            'depth': [4, 6, 8, 10],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'l2_leaf_reg': [1, 3, 5, 7, 9],
            'random_strength': [1, 2, 5],
            'auto_class_weights': ['Balanced', 'None']
    }
}

# Dictionaries to store trained instances
trained_models_baseline = {}
trained_models_tuned = {}